In [1]:
import pandas as pd

# loading train data
def load_data(filename):
    return pd.read_csv(filename)

train_data = load_data("train.csv")

In [2]:
# train_data.Sex.describe()

In [3]:
# ages
# bins = [0, 10, 18, 30, 40, 50, 60, 70, 100]
# labels = ["0", "10", "18", "30", "40", "50", "60", "70"]
# train_data["ages_range"] = pd.cut(train_data.Age, bins,
#                                   labels= labels,
#                                   include_lowest = True)

In [4]:
# feature modification
#  family numbers  - spouse + patch
# train_data["family_members"]= train_data["SibSp"] + train_data["Parch"]

In [5]:
# categories the fare 

In [6]:
def add_features(data):
#     ages -ranges
    bins = [0, 10, 18, 30, 40, 50, 60, 70, 100]
    labels = ["0", "10", "18", "30", "40", "50", "60", "70"]
    data["ages_range"] = pd.cut(data.Age, bins,
                                  labels= labels,
                                  include_lowest = True)
#     family members    
    data["family_members"]= data["SibSp"] + data["Parch"]  
    
#     fare categories.    
    f_bins = [0, 10, 20, 30, 40, 50, 75, 100, 150, 200, 300, 400, 500, 600]
    f_labels = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm' ]

    data['fare_cat'] = pd.cut(data.Fare,f_bins,
                                 labels = f_labels,
                                 include_lowest= True)
    return data

In [ ]:
# train_data = add_features(train_data)

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

In [8]:
# build the pipeline for the numerical attributes
num_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy= "median")),
    ("scaler", StandardScaler())
])

In [9]:
# build the pipeline for the categorical attributes
from sklearn.preprocessing import OneHotEncoder
cat_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("cat_encoder", OneHotEncoder(sparse=False))
])

In [10]:
# joining them both
from sklearn.compose import ColumnTransformer
num_attribs = ["Age", "SibSp", "Parch", "Fare", "family_members"]
cat_attribs = ["Pclass", "Sex", "Embarked", "ages_range", "fare_cat"]

preprocess_pipeline = ColumnTransformer([
    ("num", num_pipeline, num_attribs),
    ("cat", cat_pipeline, cat_attribs)
])

In [11]:
# raw data to numerical input features .
X_train = preprocess_pipeline.fit_transform(
    train_data[num_attribs + cat_attribs]
    )
# output to arrays

In [12]:
y_train = train_data["Survived"]

In [13]:
# let use svm 
from sklearn.svm import SVC

svm_clf = SVC(gamma ="auto")
# svm_scores = cross_val_score(svm_clf, X_train, y_train , cv = 3)
# svm_scores.mean()

In [14]:
from sklearn.model_selection import cross_val_score

In [15]:
svm_scores = cross_val_score(svm_clf, X_train, y_train , cv = 3)
svm_scores.mean()

0.819304152637486

In [16]:
svm_clf.fit(X_train, y_train)

SVC(gamma='auto')

In [17]:
# now test the test data..
test_data = load_data("test.csv")


In [18]:
testid = test_data["PassengerId"]

In [19]:
test_data = add_features(test_data)

In [20]:
X_test = preprocess_pipeline.fit_transform(
    test_data[num_attribs + cat_attribs])

In [21]:
svm_predics = svm_clf.predict(X_test)

In [22]:
svm_df = pd.DataFrame({"PassengerId":testid.values,
                   "Survived": svm_predics,
                  }
                 )

In [23]:
svm_df.to_csv("svm_submission_check.csv", index = False)